# Setup

In [1]:
from Bio.PDB import *
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
import os
import re
%matplotlib widget

In [3]:
def offset_filter(x, offset_A, offset_B):
    return x[(x[:,0] >= offset_A) & (x[:,1] >= offset_B)]

In [4]:
def min_residue_distance(resA, resB):
    distances = np.empty((len(resA), len(resB)), float)
    for i, atomA in enumerate(resA):
        for j, atomB in enumerate(resB):
            distances[i,j] = atomA - atomB
    return np.min(distances)

def min_residue_distances(chainA, chainB):
    return np.array([[min_residue_distance(resA, resB) for resB in chainB] for resA in chainA])

def get_contacts(chainA, chainB, contact_dist):
    contacts = np.argwhere(min_residue_distances(chainA, chainB) < contact_dist)

    # Correct for missing residues!
    seqA = np.array([chainA[0].id[1] - 1]+[resA.id[1] for resA in chainA])
    seqB = np.array([chainB[0].id[1] - 1]+[resB.id[1] for resB in chainB])
    idxMapA = np.cumsum((seqA[1:] - seqA[:-1]))
    idxMapB = np.cumsum((seqB[1:] - seqB[:-1]))

    return np.array([(idxMapA[contact[0]], idxMapB[contact[1]]) for contact in contacts])

In [5]:
def plot_couplings(nameA, nameB, couplings, chainA1, chainB1, chainA2=None, chainB2=None, probability = 0.99, contact_dist = 5, res_offset_A=0, res_offset_B=0, size_ratio=(1,1)):

    contact_s = 36
    coup_s = 4
    tick_spacing = 50
    contact_alpha = 0.25

    fig, ax = plt.subplots(2, 2, width_ratios=size_ratio, height_ratios=size_ratio)
    fig.set_size_inches((12, 12))
    

    ax[0,0].set_aspect('equal', 'box')
    ax[1,1].set_aspect('equal', 'box')

    ax[0,0].set_xlabel(f"{nameA} Residue")
    ax[0,0].set_ylabel(f"{nameA} Residue")
    ax[0,1].set_xlabel(f"{nameB} Residue")
    ax[0,1].set_ylabel(f"{nameA} Residue")
    ax[1,0].set_xlabel(f"{nameA} Residue")
    ax[1,0].set_ylabel(f"{nameB} Residue")
    ax[1,1].set_xlabel(f"{nameB} Residue")
    ax[1,1].set_ylabel(f"{nameB} Residue")

    coup1 = offset_filter(couplings[(couplings["segment_i"] == "A_1") & (couplings["segment_j"] == "A_1") & (couplings["probability"] >= probability)][["i","j", "probability"]].to_numpy(), res_offset_A, res_offset_A)
    coup1 = np.concatenate([coup1, coup1[:,[1,0,2]]])

    coup2 = offset_filter(couplings[(couplings["segment_i"] != couplings["segment_j"]) & (couplings["probability"] >= probability)][["j","i", "probability"]].to_numpy(), res_offset_B, res_offset_A)
    coup3 = coup2[:,[1,0,2]]

    coup4 = offset_filter(couplings[(couplings["segment_i"] == "B_1") & (couplings["segment_j"] == "B_1") & (couplings["probability"] >= probability)][["i","j", "probability"]].to_numpy(), res_offset_B, res_offset_B)
    coup4 = np.concatenate([coup4, coup4[:,[1,0,2]]])

    # Plot 1

    contacts = get_contacts(chainA1, chainA1, contact_dist)
    ax[0,0].scatter(contacts[:,0] + res_offset_A, contacts[:,1] + res_offset_A, linewidths=0, s=contact_s, c="#B7D4E8")

    if chainA2 != None:
        contacts = get_contacts(chainA1, chainA2, contact_dist)
        if len(contacts) > 0:
            ax[0,0].scatter(contacts[:,0] + res_offset_A, contacts[:,1] + res_offset_A, linewidths=0, s=contact_s, c="tab:orange", alpha=contact_alpha)

    # Plots 2 and 3

    contacts = get_contacts(chainA1, chainB1, contact_dist)
    ax[0,1].scatter(contacts[:,1] + res_offset_B, contacts[:,0] + res_offset_A, linewidths=0, s=contact_s, c="#B7D4E8")
    ax[1,0].scatter(contacts[:,0] + res_offset_A, contacts[:,1] + res_offset_B, linewidths=0, s=contact_s, c="#B7D4E8")

    if chainB2 != None:
        contacts = get_contacts(chainA1, chainB2, contact_dist)
        if len(contacts) > 0:
            ax[0,1].scatter(contacts[:,1] + res_offset_B, contacts[:,0] + res_offset_A, linewidths=0, s=contact_s, c="tab:orange", alpha=contact_alpha)
            ax[1,0].scatter(contacts[:,0] + res_offset_A, contacts[:,1] + res_offset_B, linewidths=0, s=contact_s, c="tab:orange", alpha=contact_alpha)
    elif chainA2 != None:
        contacts = get_contacts(chainB1, chainA2, contact_dist)
        if len(contacts) > 0:
            ax[0,1].scatter(contacts[:,1] + res_offset_B, contacts[:,0] + res_offset_A, linewidths=0, s=contact_s, c="tab:orange", alpha=contact_alpha)
            ax[1,0].scatter(contacts[:,0] + res_offset_A, contacts[:,1] + res_offset_B, linewidths=0, s=contact_s, c="tab:orange", alpha=contact_alpha)

    # Plot 4

    contacts = get_contacts(chainB1, chainB1, contact_dist)
    ax[1,1].scatter(contacts[:,0] + res_offset_B, contacts[:,1] + res_offset_B, linewidths=0, s=contact_s, c="#B7D4E8")

    if chainB2 != None:
        contacts = get_contacts(chainB1, chainB2, contact_dist)
        if len(contacts) > 0:
            ax[1,1].scatter(contacts[:,0] + res_offset_B, contacts[:,1] + res_offset_B, linewidths=0, s=contact_s, c="tab:orange", alpha=contact_alpha)

    axf = ax.flatten()
    annots = []
    for axi in axf:
        annot = axi.annotate("hello", xy=(0,0), xytext=(20,20),textcoords="offset points",
                            bbox=dict(boxstyle="round", fc="w"),
                            arrowprops=dict(arrowstyle="->"))
        annot.set_visible(False)
        annots.append(annot)


    annot_dict = dict(zip(axf, annots))
    coup_dict = dict(zip(axf, [coup1, coup2, coup3, coup4]))
    offset_dict = dict(zip(axf, [(res_offset_A, res_offset_A), (res_offset_B, res_offset_A), (res_offset_A, res_offset_B), (res_offset_B,res_offset_B)]))
    chain_dict = dict(zip(axf, [(chainA1, chainA1), (chainB1, chainA1), (chainA1, chainB1), (chainB1, chainB1)]))
    sc_dict = dict(zip(axf, [axi.scatter(coup_dict[axi][:,0], coup_dict[axi][:,1], s=coup_s, c="k") for axi in axf]))

    def update_annot(sc, annot, coup, offset, chains, ind):
        
        pos = sc.get_offsets()[ind["ind"][0]]
        res_x = int(coup[ind["ind"][0],0])
        res_y = int(coup[ind["ind"][0],1])
        res_pr = coup[ind["ind"][0],2]
        annot.xy = pos
        text = f"{chains[0][res_x-1 - offset[0]].get_resname()} {res_x}, {chains[1][res_y-1 - offset[1]].get_resname()} {res_y}\nProbability {res_pr:.2f}"
        annot.set_text(text)
        annot.get_bbox_patch().set_alpha(0.8)
        

    def hover(event):
        if event.inaxes in axf:
            for axi in axf:
                cont, ind = sc_dict[axi].contains(event)
                annot = annot_dict[axi]
                coup = coup_dict[axi]
                if cont:
                    update_annot(sc_dict[axi], annot, coup, offset_dict[axi], chain_dict[axi], ind)
                    annot.set_visible(True)
                    fig.canvas.draw_idle()
                else:
                    if annot.get_visible():
                        annot.set_visible(False)
                        fig.canvas.draw_idle()

    fig.canvas.mpl_connect("motion_notify_event", hover)


    ax[1,0].set_xlim(ax[0,0].get_xlim())
    ax[1,0].set_ylim(ax[1,1].get_ylim())
    ax[0,1].set_xlim(ax[1,1].get_xlim())
    ax[0,1].set_ylim(ax[0,0].get_ylim())

    fig.suptitle(f"Evolutionary Coupling Analysis for {nameA} and {nameB}\n Contact Distance < {contact_dist}Å, Coupling Probability ≥ {probability:.2f}")
    fig.legend(["Intra-Protomer Contact", "Inter-Protomer Contact", "Evolutionary Coupling"])

    for axi in axf:

        offx = offset_dict[axi][0]+1
        offy = offset_dict[axi][1]+1

        axi.set_xticks([offx, *range(int(np.ceil(offx / tick_spacing)) * tick_spacing, int(np.ceil(axi.get_xlim()[1])), tick_spacing)])
        axi.set_yticks([offy, *range(int(np.ceil(offy / tick_spacing)) * tick_spacing, int(np.ceil(axi.get_ylim()[1])), tick_spacing)])

        axi.grid()
        axi.set_axisbelow(True) 

        axi.invert_yaxis()

In [6]:
def get_multimer_tables(nameA, nameB, couplings, chainA1, chainB1, chainA2, chainB2, probability=0.99, contact_dist=5, res_offset_A=0, res_offset_B=0):
    
    coup = offset_filter(couplings[(couplings["segment_i"] != couplings["segment_j"]) & (couplings["probability"] >= probability)][["i","j", "probability"]].to_numpy(), res_offset_A, res_offset_B)

    intra_protomer_contacts = set(map(tuple,get_contacts(chainA1, chainB1, contact_dist) + [res_offset_A, res_offset_B]))

    if chainB2 != None:
        inter_protomer_contacts = set(map(tuple,get_contacts(chainA1, chainB2, contact_dist) + [res_offset_A, res_offset_B]))
    elif chainA2 != None:
        inter_protomer_contacts = set(map(tuple,get_contacts(chainB1, chainA2, contact_dist)[:,[1,0]] + [res_offset_A, res_offset_B]))

    resA = []
    resB = []
    prob = []
    contact_type = []
    for coupling in coup:

        resA.append(f"{chainA1[int(coupling[0]) - 1 - res_offset_A].get_resname()} {int(coupling[0])}")
        resB.append(f"{chainB1[int(coupling[1]) - 1 - res_offset_B].get_resname()} {int(coupling[1])}")
        prob.append(f"{coupling[2]:.2f}")

        coup_tup = (int(coupling[0]), int(coupling[1]))

        if coup_tup in intra_protomer_contacts:
            contact_type.append("Intra-Protomer")
        elif coup_tup in inter_protomer_contacts:
            contact_type.append("Inter-Protomer")
        else:
            contact_type.append("None")

    all_table = pd.DataFrame({nameA: resA, nameB: resB, "Probability (2 d.p.)": prob, "Contact Type": contact_type})
    contact_table = all_table[all_table["Contact Type"] != "None"]

    all_table.index = np.arange(1, len(all_table) + 1)
    contact_table.index = np.arange(1, len(contact_table) + 1)

    return contact_table, all_table


In [7]:
def get_monomer_tables(name, couplings, segment, chainA1, chainA2, probability=0.99, contact_dist=5, res_offset=0):
    
    coup = offset_filter(couplings[(couplings["segment_i"] == segment) & (couplings["segment_j"] == segment) & (couplings["probability"] >= probability)][["i","j", "probability"]].to_numpy(), res_offset, res_offset)

    intra_protomer_contacts = set(map(tuple,get_contacts(chainA1, chainA1, contact_dist) + [res_offset, res_offset]))

    if chainA2 != None:
        inter_protomer_contacts = set(map(tuple,get_contacts(chainA1, chainA2, contact_dist) + [res_offset, res_offset]))

    resA = []
    resB = []
    prob = []
    contact_type = []
    for coupling in coup:

        resA.append(f"{chainA1[int(coupling[0]) - 1 - res_offset].get_resname()} {int(coupling[0])}")
        resB.append(f"{chainA1[int(coupling[1]) - 1 - res_offset].get_resname()} {int(coupling[1])}")
        prob.append(f"{coupling[2]:.2f}")

        coup_tup = (int(coupling[0]), int(coupling[1]))

        if coup_tup in intra_protomer_contacts:
            contact_type.append("Intra-Protomer")
        elif coup_tup in inter_protomer_contacts:
            contact_type.append("Inter-Protomer")
        else:
            contact_type.append("None")

    all_table = pd.DataFrame({f"{name} 1": resA, f"{name} 2": resB, "Probability (2 d.p.)": prob, "Contact Type": contact_type})
    contact_table = all_table[all_table["Contact Type"] != "None"]

    all_table.index = np.arange(1, len(all_table) + 1)
    contact_table.index = np.arange(1, len(contact_table) + 1)

    return contact_table, all_table


# SleB + YpeB

In [8]:
# Consequences of signal peptide cleavage:
SleB_residue_offset = 32

In [10]:
couplings = pd.read_csv("EVCouplings Output/Bc_SleB_YpeB_all.csv")

In [13]:
parser = PDBParser()
structure = parser.get_structure("SleB_YpeB", "Models/Bc_SleB_YpeB_Dimer_of_Dimer.pdb")
[model] = [*structure.get_models()]
chainA1 = [*model["A"].get_residues()]
chainA2 = [*model["B"].get_residues()]
chainB1 = [*model["C"].get_residues()]
chainB2 = [*model["D"].get_residues()]
del parser

In [ ]:
plot_couplings("SleB", "YpeB", couplings, chainA1, chainB1, chainA2, chainB2, res_offset_A=SleB_residue_offset, probability=0.8, size_ratio=(2, 3))

In [ ]:
SleB_YpeB_contacts_table, SleB_YpeB_all_table = get_multimer_tables("SleB", "YpeB", couplings, chainA1, chainB1, chainA2, chainB2, res_offset_A=SleB_residue_offset, probability=0.6)

In [ ]:
SleB_YpeB_contacts_table

In [ ]:
SleB_YpeB_all_table

In [ ]:
YpeB_contacts_table, YpeB_all_table = get_monomer_tables("YpeB", couplings, "B_1", chainB1, chainB2, probability=0.8)

In [ ]:
YpeB_contacts_table

,YpeB 1,YpeB 2,Probability (2 d.p.),Contact Type
1,ASN 426,GLU 433,1.00,Intra-Protomer
2,VAL 371,LEU 398,1.00,Intra-Protomer
3,ALA 297,PHE 327,1.00,Intra-Protomer
4,ARG 369,GLU 406,1.00,Intra-Protomer
5,ALA 378,GLU 394,1.00,Intra-Protomer
...,...,...,...,...
153,LYS 229,GLU 238,0.83,Intra-Protomer
154,HIS 54,GLN 143,0.83,Intra-Protomer
155,LYS 229,ASN 237,0.82,Intra-Protomer
156,ASN 64,PRO 387,0.82,Inter-Protomer


In [ ]:
YpeB_contacts_table[YpeB_contacts_table["Contact Type"] == "Inter-Protomer"]

,YpeB 1,YpeB 2,Probability (2 d.p.),Contact Type
34,ASP 55,LYS 437,1.00,Inter-Protomer
49,GLY 58,TYR 421,1.00,Inter-Protomer
59,SER 67,ALA 431,1.00,Inter-Protomer
61,ALA 74,ILE 116,1.00,Inter-Protomer
116,ALA 62,LEU 416,0.95,Inter-Protomer
156,ASN 64,PRO 387,0.82,Inter-Protomer


In [ ]:
YpeB_all_table

,YpeB 1,YpeB 2,Probability (2 d.p.),Contact Type
1,ASN 426,GLU 433,1.00,Intra-Protomer
2,VAL 371,LEU 398,1.00,Intra-Protomer
3,ALA 297,PHE 327,1.00,Intra-Protomer
4,ARG 369,GLU 406,1.00,Intra-Protomer
5,ALA 378,GLU 394,1.00,Intra-Protomer
...,...,...,...,...
263,THR 200,GLN 344,0.81,None
264,LEU 32,LEU 94,0.81,None
265,VAL 15,SER 38,0.81,None
266,SER 70,GLY 88,0.80,None


# SleB + YpeB + YlaJ

In [16]:
# Consequences of signal peptide cleavage:
SleB_residue_offset = 32
YlaJ_residue_offset = 16

### YpeB - YlaJ

In [18]:
couplings = pd.read_csv("EVCouplings Output/Bc_YpeB_YlaJ_longrange.csv")

In [21]:
parser = MMCIFParser()
structure = parser.get_structure("SleB_YpeB_YlaJ", "Models/Bc_SleB_YpeB_YlaJ_Dimer_of_Trimer.cif")
[model] = [*structure.get_models()]
chainA1 = [*model["C"].get_residues()]
chainA2 = [*model["D"].get_residues()]
chainB1 = [*model["F"].get_residues()]
chainB2 = [*model["E"].get_residues()]
del parser

In [ ]:
plot_couplings("YpeB", "YlaJ", couplings, chainA1, chainB1, chainA2, chainB2, res_offset_B=YlaJ_residue_offset, probability=0.80, contact_dist=5, size_ratio=(3,2))

In [ ]:
YpeB_YlaJ_contacts_table, YpeB_YlaJ_all_table = get_multimer_tables("YpeB", "YlaJ", couplings, chainA1, chainB1, chainA2, chainB2, res_offset_B=YlaJ_residue_offset, probability=0.60)

In [ ]:
YpeB_YlaJ_contacts_table

In [ ]:
YpeB_YlaJ_all_table

### SleB - YlaJ

In [24]:
couplings = pd.read_csv("EVCouplings Output/Bc_SleB_YlaJ_longrange.csv")

# EV Couplings was unable to judge probabilities for heteromer contacts due to data insignificance
# Make optimistic guess based on other ordering
for i in range(1,len(couplings)):
    if np.isnan(couplings.iloc[i,8]):
        couplings.iloc[i,8] = couplings.iloc[i-1,8]

In [25]:
parser = MMCIFParser()
structure = parser.get_structure("SleB_YpeB_YlaJ", "Models/Bc_SleB_YpeB_YlaJ_Dimer_of_Trimer.cif")
[model] = [*structure.get_models()]
chainA1 = [*model["A"].get_residues()]
chainA2 = [*model["B"].get_residues()]
chainB1 = [*model["E"].get_residues()]
chainB2 = [*model["F"].get_residues()]
del parser

In [ ]:
plot_couplings("SleB", "YlaJ", couplings, chainA1, chainB1, chainA2, chainB2, res_offset_A=SleB_residue_offset, res_offset_B=YlaJ_residue_offset, probability=0.9)